<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/Deepseek_Llava_VLM_trl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git

Found existing installation: transformers 4.48.2
Uninstalling transformers-4.48.2:
  Successfully uninstalled transformers-4.48.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-niayizfo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-niayizfo
  Resolved https://github.com/huggingface/transformers.git to commit 6b550462139655d488d4c663086a63e98713c6b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10719728 sha256=86bfa9c6615ebea13510f3d39894af82bf6bf82999d217bec8f789255659a9c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqak38u3/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [ ]:
!pip install accelerate trl datasets wandb huggingface_hub bitsandbytes peft sentencepiece tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**NOTE: Make sure to restart the session to update the transformers library**:

## Authenticate to HF and wandb

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: smellslikeml to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Train

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", padding_side="right")

# Add <image> as a special token
special_tokens = {"additional_special_tokens": ["<image>"]}
tokenizer.add_special_tokens(special_tokens)
tokenizer.pad_token = tokenizer.eos_token

image_token_id = tokenizer.convert_tokens_to_ids("<image>")

if image_token_id == tokenizer.unk_token_id:
    print(" ERROR: <image> is still unknown after adding! Manually adding...")
    tokenizer.add_tokens(["<image>"])

tokenizer.save_pretrained("tokenizer_tmp")

('tokenizer_tmp/tokenizer_config.json',
 'tokenizer_tmp/special_tokens_map.json',
 'tokenizer_tmp/tokenizer.json')

In [ ]:
import torch
import torch.nn as nn
from transformers import (
    LlavaForConditionalGeneration, LlavaConfig, AutoModelForCausalLM, AutoTokenizer, AutoProcessor
)
from transformers.models.llava.modeling_llava import LlavaCausalLMOutputWithPast
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset, Dataset
from PIL import Image
import base64
from io import BytesIO
from tqdm import tqdm
import wandb
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from typing import Optional, List, Union, Tuple


# ===================== CONFIG CLASS =====================
class DeepSeekLlavaConfig(LlavaConfig):
    """
    Configuration class that extends LlavaConfig to use DeepSeek.
    """
    model_type = "deepseek_llava"

    def __init__(
        self,
        vision_model_name="openai/clip-vit-large-patch14",
        language_model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        vision_hidden_size=1024,
        text_hidden_size=1536,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.vision_model_name = vision_model_name
        self.language_model_name = language_model_name
        self.vision_hidden_size = vision_hidden_size
        self.text_hidden_size = text_hidden_size
        self._name_or_path = language_model_name


# ===================== MODEL CLASS =====================
class DeepSeekLlavaModel(LlavaForConditionalGeneration):
    """Vision-Language Model based on Llava, replacing the language model with DeepSeek."""

    config_class = DeepSeekLlavaConfig

    def __init__(self, config: DeepSeekLlavaConfig):
        super().__init__(config)

        # Load DeepSeek as the language model
        self.language_model = AutoModelForCausalLM.from_pretrained(config.language_model_name)
        self.config._name_or_path = config.language_model_name

        # ✅ Align vision & text embedding sizes
        #self.vision_projection = nn.Linear(1024, 1536)
        #self.multi_modal_projector = self.vision_projection
        self.vision_projection = nn.Linear(1024, 1536)
        # Instead of sharing the same layer, create a new layer with the same parameters:
        self.multi_modal_projector = nn.Linear(1024, 1536)
        self.multi_modal_projector.load_state_dict(self.vision_projection.state_dict())


        # Apply LoRA fine-tuning
        lora_config = LoraConfig(
            r=8, lora_alpha=32, lora_dropout=0.1,
            target_modules=["q_proj", "v_proj"],
            task_type="CAUSAL_LM",
        )
        self.language_model = get_peft_model(self.language_model, lora_config)
        self.language_model.print_trainable_parameters()

        # ✅ Load the correct tokenizer (after saving modifications)
        self.tokenizer = AutoTokenizer.from_pretrained("tokenizer_tmp")

        # ✅ Resize token embeddings AFTER loading the tokenizer
        self.language_model.resize_token_embeddings(len(self.tokenizer))

        # ✅ Store image token ID
        self.image_token_id = self.tokenizer.convert_tokens_to_ids("<image>")
        self.config.image_token_index = self.image_token_id  # ✅ Fixes missing image token error
        #assert self.image_token_id != self.tokenizer.unk_token_id, "🚨 <image> token is still unknown!"

        # Processor for images
        self.processor = AutoProcessor.from_pretrained(config.vision_model_name, use_fast=False)
        self.processor.image_processor.size = {"height": 336, "width": 336}
        self.processor.image_processor.crop_size = {"height": 336, "width": 336}
        self.processor.image_processor.resample = False
        #self.processor.image_token = "<img>"
        self.processor.tokenizer = self.tokenizer


    def get_input_embeddings(self):
      return self.language_model.get_input_embeddings()


    import torch.nn.functional as F

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        pixel_values: Optional[torch.FloatTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        vision_feature_layer: Optional[Union[int, List[int]]] = None,
        vision_feature_select_strategy: Optional[str] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        logits_to_keep: Union[int, torch.Tensor] = 0,
        image_sizes: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Union[Tuple, LlavaCausalLMOutputWithPast]:
        # Set default values from config if not provided
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        vision_feature_layer = vision_feature_layer if vision_feature_layer is not None else self.config.vision_feature_layer
        vision_feature_select_strategy = (
            vision_feature_select_strategy if vision_feature_select_strategy is not None else self.config.vision_feature_select_strategy
        )

        # Exactly one of input_ids or inputs_embeds must be provided.
        if (input_ids is None) == (inputs_embeds is None):
            raise ValueError("You must specify exactly one of input_ids or inputs_embeds")
        if pixel_values is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both pixel_values and inputs_embeds at the same time; choose one.")

        # If no inputs_embeds provided, derive them from input_ids
        if inputs_embeds is None:
            inputs_embeds = self.get_input_embeddings()(input_ids)

        # Process image features if pixel_values provided
        if pixel_values is not None:
            # Get image features from the vision tower and project them.
            image_features = self.get_image_features(
                pixel_values=pixel_values,
                vision_feature_layer=vision_feature_layer,
                vision_feature_select_strategy=vision_feature_select_strategy,
                image_sizes=image_sizes,
            )
             # ✅ Ensure `image_features` matches input embedding dtype
            image_features = image_features.to(self.get_input_embeddings().weight.dtype)

            # Instead of using a simple masked_scatter, call the merging helper.
            # This function takes:
            # - image_features: the projected vision features
            # - inputs_embeds: the original text embeddings
            # - input_ids: to identify which tokens are image placeholders
            # - attention_mask: the original attention mask
            # - labels: if available
            #
            # It returns new embeddings, attention mask, (possibly updated) labels, and position ids.
            inputs_embeds, attention_mask, labels, position_ids = self._merge_input_ids_with_image_features(
                image_features, inputs_embeds, input_ids, attention_mask, labels
            )
            # Note: this helper handles many alignment issues (e.g. left padding, position shifting, etc.)

        # Forward pass through the language model
        outputs = self.language_model(
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
            logits_to_keep=logits_to_keep,
            **kwargs,
        )

        logits = outputs[0]

        # Compute loss if labels are provided. (Shift logits and labels accordingly)
        loss = None
        if labels is not None:
            if attention_mask is not None:
                shift_attention_mask = attention_mask[:, -(logits.shape[1] - 1):].to(logits.device)
                shift_logits = logits[..., :-1, :][shift_attention_mask != 0].contiguous()
                shift_labels = labels[..., 1:][shift_attention_mask.to(labels.device) != 0].contiguous()
            else:
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
            loss = torch.nn.functional.cross_entropy(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1).to(shift_logits.device)
            )

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return LlavaCausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values if hasattr(outputs, "past_key_values") else None,
            hidden_states=outputs.hidden_states if hasattr(outputs, "hidden_states") else None,
            attentions=outputs.attentions if hasattr(outputs, "attentions") else None,
            image_hidden_states=image_features if pixel_values is not None else None,
        )


# ===================== DATASET FORMATTING =====================
def format_data_openspaces(sample):
    """
    Converts a sample into the correct format for training using the model's chat template.
    """
    formatted_messages = []
    image_data = None

    for turn in sample["new_messages_final"]:
        if turn["role"] == "user":
            has_image = any(item["type"] == "image" for item in turn["content"])
            text_content = ""

            for item in turn["content"]:
                if item["type"] == "text":
                    text_content += item["text"] + " "  # Ensure spacing
                elif item["type"] == "image":
                    text_content = "<image>\n" + text_content  # ✅ **Force <image> token**

            formatted_messages.append({"role": "user", "content": text_content.strip()})

            # ✅ Ensure the image is correctly processed
            for item in turn["content"]:
                if item["type"] == "image":
                    try:
                        image_bytes = base64.b64decode(item["image"])
                        image_data = Image.open(BytesIO(image_bytes)).convert("RGB").resize((336, 336))
                    except Exception as e:
                        print(f"Error processing image: {e}")
                        continue

        elif turn["role"] == "assistant":
            formatted_messages.append({"role": "assistant", "content": turn["content"][0]["text"]})

    return {"messages": formatted_messages, "image": image_data}

# ===================== DATASET =====================

openspaces_dataset_id = "salma-remyx/OpenSpaces-qwen-vlm-instruct"
train_dataset_openspaces = load_dataset(openspaces_dataset_id, split="train")

dataset_openspaces = [format_data_openspaces(sample) for sample in tqdm(train_dataset_openspaces, desc="Formatting Data")]
train_dataset = Dataset.from_list(dataset_openspaces)

# ===================== COLLATE FUNCTION =====================
def collate_fn(examples):
    """
    Data collator for training with images and text, applying the chat template manually.
    """
    texts = []
    images = []

    for example in examples:
        if "messages" not in example:
            raise KeyError(f"Missing 'messages' key in example: {example}")

        formatted_text = ""
        for message in example["messages"]:
            role = message["role"].upper() + ": "

            # 🔹 **Fix: Ensure `content` is treated as a string**
            text_content = message["content"].strip()

            # 🔥 Ensure <image> token is included properly
            if "<image>" in text_content:
                text_content = text_content.replace("<image>", " <image> ")  # Add spacing around it

            formatted_text += f"{role}{text_content}\n"

        texts.append(formatted_text)

        # ✅ Ensure images exist and resize correctly
        if "image" in example and example["image"] is not None:
            img = example["image"].convert("RGB").resize((336, 336))  # ✅ Explicit resize
        else:
            img = Image.new("RGB", (336, 336))  # Placeholder blank image

        images.append(img)

    # ✅ Process batch with processor
    batch = vlm_model.processor(text=texts, images=images, return_tensors="pt", padding=True)

    # ✅ Handle labels correctly
    labels = batch["input_ids"].clone()
    labels[labels == vlm_model.tokenizer.pad_token_id] = -100
    labels[labels == vlm_model.image_token_id] = -100  # Ignore image token in loss computation

    batch["labels"] = labels
    return batch


# ===================== TRAINING SETUP =====================
training_args = SFTConfig(
    output_dir="vlm-finetune",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    bf16=True,
    report_to="wandb",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

wandb.init(project="vlm-finetune", name="vlm-finetune", config=training_args)

config = DeepSeekLlavaConfig()
vlm_model = DeepSeekLlavaModel(config)


trainer = SFTTrainer(
    model=vlm_model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    processing_class=vlm_model.tokenizer)

trainer.train()
trainer.save_model(training_args.output_dir)
vlm_model.language_model.save_pretrained(f"{training_args.output_dir}/lora")
vlm_model.tokenizer.save_pretrained(training_args.output_dir)

README.md:   0%|          | 0.00/886 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/71 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/71 [00:00<?, ?it/s]

train-00000-of-00071.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00001-of-00071.parquet:   0%|          | 0.00/319M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,2.792000
1000,2.187000
1500,1.897000
2000,1.694400
2500,1.607300
3000,1.539600


Step,Training Loss
500,2.792000
1000,2.187000
1500,1.897000
2000,1.694400
2500,1.607300
3000,1.539600
3500,1.517500
4000,1.507900
4500,1.445600
5000,1.464600


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })